# Experimentos com o gerenciamento de grid

A ideia é calcular o fatiamento de uma imagem gigante, e depois localizar a posição exata em um dos quadrantes.

Com isso será possível fazer o processamento de uma imagem maior do que a memória ram disponível, dividindo ela em diversos quadrantes.

Posteriormente será estudado uma forma de unir todos os quadrantes na imagem final, mesmo que não tenha ram suficiente para montar da forma convencional.

In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import math
import h5py

In [3]:
# simulando uma imagem gigante, mantendo um aspect ratio de 16:9
ratio = 16 / 9
# imgWidth = 137438953472 # teste de imagem maior do que a ram disponível
imgWidth = 1024 # teste básico para cálculo 
                 #(já chegou a processar um array de coordenadas para imagem de 1.500.000 pixels)
imgHeight = int(imgWidth / ratio)

imgHeight

576

In [4]:
# criado uma sequencia aleatória para gerar uma imagem de testes com as dimensões exatas para o teste
# o valor 3, refere aos canais de cores BGR (azul, verde, vermelho)
# https://docs.scipy.org/doc/numpy-1.13.0/user/basics.types.html
# img = np.random.random([imgHeight, imgWidth, 3]) * 255 # o mesmo que uint8
# img = img.astype(int)
# img[:] = img.mean(axis=-1,keepdims=1) # convertendo para escala de cinza
# img

In [5]:
# plt.imshow(img)

utilizando como base a imagem carregada e suas dimensões, será feita uma função para separar as partes da imagem em uma grid.

In [6]:
# esta função retorna um vetor com todas as posições x e y de cada quadrante
# utilizando como base as dimensões informadas
def dividirEmQuadrantes( largura: int, altura: int, quadranteMinimo: int = 128 ):
    # o quadrante não pode ser maior do que a largura ou altura da imagem
    if( quadranteMinimo < largura | quadranteMinimo < altura ):
        quadranteMinimo = ( altura, largura )[largura < altura] # teste if ternario (falseValue, trueValue)[test]
    
    # o quadrante mínimo não pode ser menor do que 2 pixels
    if( quadranteMinimo < 2 ):
        raise Exception("O valor do quadrante mínimo deve ser maior ou igual a 2")
    
    # a largura não pode ser menor do que 2 pixels
    if( largura < 2 ):
        raise Exception("A largura mínima deve ser maior ou igual a 2")
    
    # a altura mínima não pode ser menor do que 2 pixels
    if( altura < 2 ):
        raise Exception("A altura mínima deve ser maior ou igual a 2")
    
    # criando a divisão mínima de largura
    quadranteMinimoLargura = quadranteMinimo
    
    while largura % quadranteMinimoLargura != 0:
        quadranteMinimoLargura = quadranteMinimoLargura + 1
    
    # criando a divisão mínima de altura
    quadranteMinimoAltura = quadranteMinimo
    
    while altura % quadranteMinimoAltura != 0:
        quadranteMinimoAltura = quadranteMinimoAltura + 1
    
    # verificando as divisões da largura
    divisoesLargura = math.floor( largura / quadranteMinimoLargura )
    
    # verificando as divisões da altura
    divisoesAltura = math.floor( altura / quadranteMinimoAltura )
    
    # calculando o total de divisões
    totalDivisoes = divisoesLargura * divisoesAltura
    
    # montando um vetor de saida, onde cada item possui os valores de x1, y1, x2, y2
    gridSaida = np.zeros(
        (
            totalDivisoes, # quantidade de itens no maior nível, neste caso são todas as divisões
            4 # quantidade de colunas no menor vível (x1, y1, x2, y2)
        ), 
        dtype=int
    )
    
    # calculando o x e y para cada posição, percorrendo cada linha (altura) e cada coluna (largura)
    # acessando o indice com a função que enumera as posições
    
    indicePrincipal = 0 # controle do indice do grid
    
    for indexAltura in range(0, divisoesAltura):
        for indexLargura in range(0, divisoesLargura):
        
            x1 = indexLargura * quadranteMinimoLargura
            y1 = indexAltura * quadranteMinimoAltura
            
            x2 = ( indexLargura + 1 ) * quadranteMinimoLargura
            y2 = ( indexAltura + 1 ) * quadranteMinimoAltura
            
            gridSaida[ indicePrincipal ][0] = x1
            gridSaida[ indicePrincipal ][1] = y1
            gridSaida[ indicePrincipal ][2] = x2
            gridSaida[ indicePrincipal ][3] = y2
            
            indicePrincipal = indicePrincipal + 1
                
    
    return gridSaida
    
    
# quanto maior a fatia, mais leve será o array, porém mais pesado será o processamento de cada fatia
gridImagem = dividirEmQuadrantes(imgWidth, imgHeight, 128)

gridImagem

array([[   0,    0,  128,  144],
       [ 128,    0,  256,  144],
       [ 256,    0,  384,  144],
       [ 384,    0,  512,  144],
       [ 512,    0,  640,  144],
       [ 640,    0,  768,  144],
       [ 768,    0,  896,  144],
       [ 896,    0, 1024,  144],
       [   0,  144,  128,  288],
       [ 128,  144,  256,  288],
       [ 256,  144,  384,  288],
       [ 384,  144,  512,  288],
       [ 512,  144,  640,  288],
       [ 640,  144,  768,  288],
       [ 768,  144,  896,  288],
       [ 896,  144, 1024,  288],
       [   0,  288,  128,  432],
       [ 128,  288,  256,  432],
       [ 256,  288,  384,  432],
       [ 384,  288,  512,  432],
       [ 512,  288,  640,  432],
       [ 640,  288,  768,  432],
       [ 768,  288,  896,  432],
       [ 896,  288, 1024,  432],
       [   0,  432,  128,  576],
       [ 128,  432,  256,  576],
       [ 256,  432,  384,  576],
       [ 384,  432,  512,  576],
       [ 512,  432,  640,  576],
       [ 640,  432,  768,  576],
       [ 7

In [7]:
# salvando em um arquivo hdf5 para utilizar posteriormente
file = h5py.File("grid.hdf5",mode="w")
group = file.create_group("coordenadas")
group.create_dataset('dataset_1', data=gridImagem)
file.close()

testando a saída do grid na imagem

In [8]:
imgTesteGrid = np.zeros((imgHeight, imgWidth ,3), np.uint8)

for quadrante in gridImagem:
    cv2.rectangle(imgTesteGrid, (quadrante[0], quadrante[1]), (quadrante[2], quadrante[3]), (0, 255, 255), 1)
    
#plt.imshow(imgTesteGrid)

# a imagem processada e salva pelo cv2 possui dpi baixo
# porém é extremamente eficiente para grandes imagens
# já o PIL trava ao carregar um array numpy muito grande
# sendo assim, todo o processo será feito com cv2 e numpy, e apenas as fatias serão salvas separadamente
# e incrementado o DPI com o PIL
cv2.imwrite('testeGrid.png', imgTesteGrid)

True

In [9]:
# lendo o arquivo hdf5
hf = h5py.File("grid.hdf5", mode="r")
dados = hf['coordenadas']['dataset_1']

gridImagem = dados.value
hf.close()

### Área de colisão

Uma vez que temos todo o grid mapeado, tentaremos detectar em quais quadrantes uma imagem colide.

Por exemplo, se em uma grid com quadrantes de 512 px, se em qualquer lugar for colocada uma nova imagem por cima (retângulo sempre), em quais quadrantes haverá uma _colisão_?

Lembrando que a ideia é processar (renderizar) apenas os quadrantes onde a imagem adicional colide.

In [25]:
# esta função recebe o grid a ser testado,
# a posição do novo elemento tuple(x, y)
# o tamanho do elemento tuple(w, h)
# com isso ele retorna um dicionario de dados
# com os indices do grid e suas coordenadas que possuem colisão
def detectarColisaoGrid( gridTeste: list, posicaoElemento: tuple, tamanhoElemento: tuple ):
    #print( gridTeste, posicaoElemento, tamanhoElemento )
    
    # 50% para cada lado quando posicionar
    # lambda é uma função anônima
    # https://www.python-course.eu/lambda.php
    # neste caso cada elemento do tuple original
    # é mapeado para o argumanto x da função lambda
    # e ao final retornado um novo tuple calculado
    tamanhoElementoPosicional = tuple( map(lambda x: x * 0.5, tamanhoElemento) )
    
    # calculando os quadrantes com base na posição do elemento
    # considerando seu tamanho
    c_x1 = posicaoElemento[0] - tamanhoElementoPosicional[0]
    c_y1 = posicaoElemento[1] - tamanhoElementoPosicional[1]
    c_x2 = posicaoElemento[0] + tamanhoElementoPosicional[0]
    c_y2 = posicaoElemento[1] + tamanhoElementoPosicional[1]
    
    #print(c_x1, c_y1, c_x2, c_y2)
    #print(gridTeste)
    
    retorno = dict()
    
    # agora verificando qual quadrante coincide com o colisor
    for index, quadrante in enumerate(gridTeste):
        x1, y1, x2, y2 = quadrante
        
#        print(x1, x2, c_x1, c_x2)
        
        colideX = False
    
        # colisão no eixo x
        if c_x1 >= x1 and c_x1 <= x2:
            print( "Colisao X", {index: quadrante} )
            
            # @TODO: Continuar a verificação daqui
            if c_x2 >= x2:
                print( "Colisao X", {index: quadrante} )
    
    return retorno
            


# testando com coorenadas quaisquer
novoElementoPosicao = ( 444, 167 )
novoElementoTamanho = ( 545, 789 )

detectarColisaoGrid(gridImagem, novoElementoPosicao, novoElementoTamanho)

Colisao X {1: array([128,   0, 256, 144])}
Colisao X {1: array([128,   0, 256, 144])}
Colisao X {9: array([128, 144, 256, 288])}
Colisao X {9: array([128, 144, 256, 288])}
Colisao X {17: array([128, 288, 256, 432])}
Colisao X {17: array([128, 288, 256, 432])}
Colisao X {25: array([128, 432, 256, 576])}
Colisao X {25: array([128, 432, 256, 576])}


{}